In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [2]:
!pip install snowflake-sqlalchemy

     |████████████████████████████████| 13.2 MB 7.8 MB/s 
     |████████████████████████████████| 1.9 MB 10.5 MB/s 
     |████████████████████████████████| 192 kB 11.4 MB/s 
     |████████████████████████████████| 133 kB 11.4 MB/s 
     |████████████████████████████████| 345 kB 11.0 MB/s 
     |████████████████████████████████| 85 kB 2.8 MB/s 
     |████████████████████████████████| 138 kB 11.1 MB/s 
     |████████████████████████████████| 45 kB 2.2 MB/s 
  Attempting uninstall: chardet
    Found existing installation: chardet 4.0.0
    Uninstalling chardet-4.0.0:
      Successfully uninstalled chardet-4.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyterlab-git 0.11.0 requires nbdime<2.0.0,>=1.1.0, but you have nbdime 2.1.0 which is incompatible.
aiobotocore 1.3.0 requires botocore<1.20.50,>=1.20.49, but you have botocore 1.20.53 which is incompatible.

# Part 2 - The "Better" Metric

Usually, Linear Regression/Time Series Forecast errors are measured by RMSE/MAE. 

RMSE is used if you want to penalize errors significantly. 10 is way worse than 5 when measuring difference between predictions and actual values.

MAE is used when you have outliers in your trained (actual) data and need to take them into consideration when assessing your forecast.

If we are measuring model performance by loss function then RMSE wins out because it is easier to visualize where you need to optimize the prediction model to achieve 0 difference to actual values. RMSE would show concave loss function whereas MAE would show relatively shallow curve where optimization point is not as easily discernible.

# Part 3 - The App

SQLAlchemy vs Snowflake Connector vs custom Python script: pandas_to_snowflake.py. I tried all three and SQLAlchemy was the one that had the least amount of code to excute and gave me no problems.

In [3]:
from sqlalchemy import create_engine
from snowflake.sqlalchemy import URL


url = URL(
    account = '',
    user = '',
    password = '',
    warehouse = '',
    database = '',
    schema = '')

engine = create_engine(url)

connection = engine.connect()

/opt/conda/lib/python3.7/site-packages/snowflake/connector/vendored/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.10) or chardet (4.0.0) doesn't match a supported version!
  RequestsDependencyWarning)


In [4]:
query = 'select "a"."settlement_time", "a"."actual_price", "f"."p50" as "fluence_p50", "m"."p50" as "market_p50", abs("f"."p50"-"a"."actual_price") as "f_diff_a", abs("m"."p50"-"a"."actual_price") as "m_diff_a" from "CASESTUDY"."CASESTUDY_MINDY"."actual_prices" as "a" join "CASESTUDY"."CASESTUDY_MINDY"."fluence_forecast" as "f" on ("a"."settlement_time" = "f"."settlement_time") join "CASESTUDY"."CASESTUDY_MINDY"."market_forecast" as "m" on ("a"."settlement_time" = "m"."settlement_time") '
df = pd.read_sql_query(query, connection)
df

,settlement_time,actual_price,fluence_p50,market_p50,f_diff_a,m_diff_a
0,2020-09-30 18:10:00,0.040440,0.040000,0.040440,0.000440,0.000000
1,2020-09-30 19:45:00,0.045168,0.046548,0.045300,0.001379,0.000132
2,2020-09-30 20:15:00,0.069786,0.071288,0.046329,0.001502,0.023457
3,2020-09-30 21:20:00,0.045108,0.047568,0.042326,0.002460,0.002782
4,2020-09-30 21:50:00,0.042650,0.044261,0.043986,0.001611,0.001336
...,...,...,...,...,...,...
105403,2020-06-30 13:20:00,0.039558,0.039674,0.040653,0.000116,0.001095
105404,2020-06-30 13:40:00,0.041462,0.041390,0.043428,0.000072,0.001966
105405,2020-06-30 14:40:00,0.045450,0.044650,0.045361,0.000800,0.000089
105406,2020-06-30 16:55:00,0.009282,0.010438,0.029739,0.001156,0.020458


On top of the base table, I decided to add f_diff_a and m_diff_a (fluence/market difference from actual prices) to show at row level which forecast had closest to 0 values. Those closest to 0 had best forecast.

In [5]:
df.info() #Look at data types to see if casting needed before calculations performed

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105408 entries, 0 to 105407
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   settlement_time  105408 non-null  datetime64[ns]
 1   actual_price     105408 non-null  float64       
 2   fluence_p50      105408 non-null  float64       
 3   market_p50       105408 non-null  float64       
 4   f_diff_a         105408 non-null  float64       
 5   m_diff_a         105408 non-null  float64       
dtypes: datetime64[ns](1), float64(5)
memory usage: 4.8 MB


### Data Aggregated at Different Levels of Granularity

I decided to aggregate the data since trends/spikes tend to pop out with condensed time axis.

In [6]:
#MONTH LEVEL
df_m = df.groupby(pd.Grouper(key='settlement_time', freq='1M')).sum() # groupby each 1 month
df_m.index = df_m.index.strftime('%B')
df_m.reset_index(level=0, inplace=True)
df_m

,settlement_time,actual_price,fluence_p50,market_p50,f_diff_a,m_diff_a
0,January,757.631669,688.906162,612.360566,288.597987,448.619325
1,February,522.478689,585.240967,281.962699,290.543207,472.842122
2,March,417.486341,430.491714,335.235005,157.234202,287.529314
3,April,285.298807,313.350951,286.884745,78.772235,102.437786
4,May,334.034443,346.652974,321.181177,63.783120,94.336727
5,June,435.086640,434.084710,470.570613,104.615195,172.536026
6,July,531.019888,538.218239,509.707834,96.705082,118.774313
7,August,405.785577,400.591088,367.163282,120.266386,168.133378
8,September,132.659816,214.204248,-87.072328,185.022809,432.015392
9,October,294.499021,332.944260,192.167906,162.569119,287.450400


In [7]:
#DAY LEVEL
df_d = df.groupby(pd.Grouper(key='settlement_time', freq='1D')).sum() # groupby each 1 day
df_d.index = df_d.index.strftime('%D')
df_d.reset_index(level=0, inplace=True)
df_d

,settlement_time,actual_price,fluence_p50,market_p50,f_diff_a,m_diff_a
0,01/01/20,15.979213,15.675179,16.194366,0.880997,1.109608
1,01/02/20,17.425347,17.175804,17.769880,1.200887,1.588379
2,01/03/20,17.612544,17.345383,16.735262,1.062282,1.703308
3,01/04/20,3.953061,8.728061,6.702663,9.373039,22.445173
4,01/05/20,9.795817,11.102372,8.305152,2.824898,4.315915
...,...,...,...,...,...,...
361,12/27/20,-0.518934,-0.005902,0.551613,5.068375,6.247147
362,12/28/20,1.844367,3.088712,1.587054,3.975789,4.639342
363,12/29/20,6.908514,6.524328,6.399399,1.792117,2.081073
364,12/30/20,2.953132,4.033863,-5.978796,3.651136,9.859740


In [8]:
#Send aggregations to DWH
df_m.to_sql('month_aggregate', con=engine, if_exists='replace', index=False)
df_d.to_sql('day_aggregate', con=engine, if_exists='replace', index=False)

# Calculate "Better" Metric

In [9]:
#RMSE for Fluence predictions
y_true = df['actual_price']
y_pred = df['fluence_p50']
mse = mean_squared_error(y_true, y_pred)
f_rmse = mse**.5
f_rmse

0.20397111794724757

In [10]:
#MAE for Fluence predictions
y_true = df['actual_price']
y_pred = df['fluence_p50']
f_mae = mean_absolute_error(y_true, y_pred)
f_mae

0.017422990170856956

In [11]:
#RMSE for market predictions
y_true = df['actual_price']
y_pred = df['market_p50']
mse = mean_squared_error(y_true, y_pred)
m_rmse = mse**.5
m_rmse

0.27224819556706203

In [12]:
#MAE for market predictions
y_true = df['actual_price']
y_pred = df['market_p50']
m_mae = mean_absolute_error(y_true, y_pred)
m_mae

0.02947188083997419

In order to determine which model was better, need to choose one metric and compare between forecast and market models. As stated earlier, RMSE was chosen as metric to use. So MAE is just used as a comparison. What is considered better performing model will have the lowest calculation for metric(s) of interest. In both cases (RMSE and MAE), Fluence forecast has lowest results. Hence, it is the better forecast compared to market forecast.

In [13]:
horizon = df['settlement_time'].max() - df['settlement_time'].min()
horizon

Timedelta('365 days 23:55:00')

In [14]:
d = {'Forecast_Source': ['Fluence', 'Market'],'Horizon_Days': [horizon.days, horizon.days], 'RMSE': [f_rmse, m_rmse], 'MAE': [f_mae, m_mae]}
metric_df =  pd.DataFrame(data=d)
metric_df = metric_df.sort_values(by = ['RMSE'])
metric_df

,Forecast_Source,Horizon_Days,RMSE,MAE
0,Fluence,365,0.203971,0.017423
1,Market,365,0.272248,0.029472


In [15]:
#store metric_df in DWH
metric_df.to_sql('metric', con=engine, if_exists='replace', index=False)